<style>
.jp-RenderedHTMLCommon pre {
    white-space: pre;
    overflow-x: auto;
    word-wrap: normal;
}
.jp-Cell-inputWrapper {
    overflow-x: auto;
}
</style>

<style>
.jp-RenderedHTMLCommon pre {
    white-space: pre;
    overflow-x: auto;
    word-wrap: normal;
}
.jp-Cell-inputWrapper {
    overflow-x: auto;
}
@media print {
  pre, code {
    white-space: pre !important;
    overflow-x: auto !important;
    word-wrap: normal !important;
  }
}
</style>

# Portada

## Análisis del sector funerario en Estados Unidos a través de datos abiertos (2013–2022)

Este proyecto realiza un análisis exploratorio de datos del sector funerario en Estados Unidos utilizando técnicas de web scraping, procesamiento de datos y visualización estadística. La información fue obtenida a través de la API de Data USA, considerando variables como número de establecimientos, población empleada y salario promedio por estado y año.


1. [Portada](#portada)  
2. [Introducción](#introducción)  
3. [Instalación e importación de bibliotecas](#instalación-e-importación-de-bibliotecas)  
4. [Webscraping de datos](#webscraping-de-datos)  
   - 4.1 [Acceso y permisos](#acceso-y-permisos)  
   - 4.2 [Extracción de datos desde Data USA](#extracción-de-datos-desde-data-usa)  
5. [Limpieza y combinación de datos](#limpieza-y-combinación-de-datos)  
   - 5.1 [Conversión de tipos y manejo de valores nulos](#conversión-de-tipos-y-manejo-de-valores-nulos)  
   - 5.2 [Estructuración por periodos de análisis](#estructuración-por-periodos-de-análisis)  
6. [Exportación a CSV](#exportación-a-csv)  
7. [Estadística descriptiva](#estadística-descriptiva)  
   - 7.1 [Análisis general por periodo](#análisis-general-por-periodo)  
   - 7.2 [Interpretación de tendencias](#interpretación-de-tendencias)  
8. [Visualización de datos](#visualización-de-datos)  
   - 8.1 [Empleados por negocio](#empleados-por-negocio)  
   - 8.2 [Evolución del salario promedio](#evolución-del-salario-promedio)  
   - 8.3 [Promedio de negocios funerarios por año](#promedio-de-negocios-funerarios-por-año)  
9. [Análisis de correlaciones](#análisis-de-correlaciones)  
10. [Segmentación por población empleada](#segmentación-por-población-empleada)  
11. [Conclusiones](#conclusiones)  
12. [Recomendaciones](#recomendaciones)  
13. [Limitaciones](#limitaciones)
14. [Anexos](#anexos)  
   - 14.1 [Código fuente](#código-fuente)  
   - 14.2 [Archivos CSV](#archivos-csv)
   - 14.3 [Recursos externos](#recursos-externos)
   - 14.4 [Manejo de archivos](#manejo-de-archivos)

# Introducción

Este análisis fue realizado con el objetivo de brindar información estratégica a una startup interesada en ingresar al sector funerario en Estados Unidos. A través del uso de técnicas de web scraping, limpieza y análisis de datos, se recopilaron y examinaron indicadores clave como el número de establecimientos funerarios, el tamaño de la fuerza laboral empleada y el salario promedio por estado, durante el periodo comprendido entre 2013 y 2022. El estudio se enfoca en identificar patrones geográficos, tendencias salariales y posibles correlaciones entre las variables, lo que permite a la empresa comprender mejor la estructura del sector y detectar oportunidades de entrada en mercados con alta concentración operativa o condiciones laborales favorables. Esta información es especialmente relevante para diseñar una estrategia de expansión informada, alineada con las realidades económicas y demográficas de cada estado. El análisis también considera las limitaciones de los datos disponibles, propone segmentaciones relevantes, y visualiza hallazgos clave mediante gráficas descriptivas. Todo esto con el propósito de ofrecer una herramienta analítica útil y accionable para la toma de decisiones estratégicas.


# Instalación e importación de bibliotecas

In [402]:
# Instalar las bibliotecas necesarias para el web scraping y análisis de datos
# Ejecutar este comando en la terminal si no estás usando Jupyter Notebook:
# pip install requests bs4 pandas numpy matplotlib seaborn

In [403]:
# Incluir las bibliotecas necesarias
import requests
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as ax
import seaborn as sns
from bs4 import BeautifulSoup
from bs4 import NavigableString
from requests.exceptions import HTTPError

In [346]:
# Cambiar el directorio de trabajo a la carpeta del proyecto
os.chdir("/wd/")

# Webscraping de datos

## Acceso y permisos

No existe ningún `robots.txt` pero según https://datausa.io/about/usage:

"Data USA is an aggregate visualization engine with public datasets from many different sources including those from official US departments as well as educational institutions. We encourage the use of the site as a means of informational and educational purposes. All of the content on the site is presented under a GNU Affero General Public License v3.0 (GPLv3). You can copy, download or print content for your own use, and you can also include excerpts from Data USA, databases and multimedia products in your own documents, presentations, blogs, websites and teaching materials, provided that suitable acknowledgment of Data USA as source is given."

## Extracción de datos desde Data USA

In [225]:
# Scrape datos de funerarias en Estados Unidos
def scrape_funeral(medida: str):
    base_url = "http://gary-app.datausa.io/api/data"

    parametros = {
        "drilldowns": "Year,State",
        "measures": medida,
        "Industry Group": "8122",
    }

    if medida in ["Average Wage", "Total Population"]:
        parametros["Workforce Status"] = "True"
    
    try:
        respuesta = requests.get(base_url, params=parametros)
        respuesta.raise_for_status()
        print("Solicitud exitosa.")
    except HTTPError as error_http:
        print(f"Error HTTP: {error_http}")
        return None
    except Exception as otro_error:
        print(f"Otro error: {otro_error}")
        return None

    datos = respuesta.json().get("data", [])

    return datos

In [ ]:
# Hacer scraping para el número de negocios, número de trabajadores y salario promedio por estado por anio
num_negocios = scrape_funeral("Total Number of Establishments")
num_trabajadores = scrape_funeral("Total Population")
avg_wage = scrape_funeral("Average Wage")

# Limpieza y combinación de datos

In [ ]:
# Determinar los keys para crear los DataFrames
print("Número de negocios " + str(num_negocios[0].keys()))
print("Número de trabajadors " + str(num_trabajadores[0].keys()))
print("Salário promedio " + str(avg_wage[0].keys()))

In [ ]:
# Convertir a DataFrames y seleccionar columnas relevantes
df_negocios = pd.DataFrame(num_negocios)[["State", "Year", "Total Number of Establishments"]]
df_trabajadores = pd.DataFrame(num_trabajadores)[["State", "Year", "Total Population"]]
df_salario = pd.DataFrame(avg_wage)[["State", "Year", "Average Wage"]]

# Combinar con unión externa para incluir todos los estados y todos los anios
df_combinado = df_negocios.merge(df_trabajadores, on=["State", "Year"], how="outer")
df_combinado = df_combinado.merge(df_salario, on=["State", "Year"], how="outer")

# Filtrar por anios y estado
df_combinado = df_combinado.sort_values(by=["State", "Year"]).reset_index(drop=True)

## Conversión de tipos y manejo de valores nulos

In [ ]:
# Verificar los tipos de datos de las columnas
df_combinado.dtypes

Ya que "Year" aparece como object (texto), es importante convertirla a tipo numérico para análisis, ordenamiento y filtrado temporal.

In [ ]:
# Convertir la columna "Year" a tipo numérico y manejar errores
df_combinado["Year"] = pd.to_numeric(df_combinado["Year"], errors="coerce").astype("Int64")
df_combinado.dtypes

In [ ]:
# Determinar el número de valores nulos en cada columna
df_combinado.isna().sum()

In [ ]:
# Investiguemos
df_combinado.head(21)

Aquí se puede notar que todos los estados solo tienen información sobre el número de negocios desde 2013 al 2016, sobre el número de trabajadores de 2017 al 2022 y sobre el salário promedio desde 2014 al 2022, excepto en el caso de Alaska, el cual le falta el número de trabajadores y el sálario promedio desde 2014 al 2016. Entonces, en vez de quitar todas las filas inmediatamente, vale la pena crear ventanas distintas para el análisis.

In [ ]:
# Quitamos la fila con el estado #null
df_combinado = df_combinado[df_combinado["State"] != "#null"]
df_combinado.head(5)

## Estructuración por periodos de análisis

In [413]:
# Usa este subconjunto para hacer análisis de correlación entre: número de negocios, población, salario promedio
df_2014_2016 = df_combinado[df_combinado["Year"].between(2014, 2016) 
                & df_combinado[["Total Number of Establishments", "Total Population", "Average Wage"]]
                .notna()
                .all(axis=1)]

# Usa este subconjunto para analizar tendencias como: evolución del salario relación entre población y salario
df_2017_2022 = df_combinado[df_combinado["Year"].between(2017, 2022) 
                & df_combinado[["Total Population", "Average Wage"]]
                .notna()
                .all(axis=1)].drop(["Total Number of Establishments"], axis=1)

# Útil para observar la presencia de negocios por estado y año, aunque sin poder compararla con población o salarios.
df_2013_2016 = df_combinado[df_combinado["Year"].between(2013, 2016) 
                & df_combinado[["Total Number of Establishments"]]
                .notna()
                .all(axis=1)].drop(["Total Population", "Average Wage"], axis=1)

# Exportación a CSV

In [347]:
# Exportar los DataFrames a archivos CSV
df_combinado.to_csv("funerarias_estados_unidos.csv", index=False)
df_2014_2016.to_csv("funerarias_2014_2016.csv", index=False)
df_2017_2022.to_csv("funerarias_2017_2022.csv", index=False)
df_2013_2016.to_csv("funerarias_2013_2016.csv", index=False)

# Estadística descriptiva

## Análisis general por periodo

In [ ]:
# Corramos estadística descriptiva para las ventanas
print("Estadística descriptiva para 2014-2016:")
print(df_2014_2016.describe())
print("\nEstadística descriptiva para 2017-2022:")
print(df_2017_2022.describe())
print("\nEstadística descriptiva para 2013-2016:")
print(df_2013_2016.describe())

## Interpretación de tendencias

Entre 2014 y 2022, el sector funerario en Estados Unidos muestra estabilidad en el tamaño del empleo (alrededor de 2,700 personas por estado) pero un crecimiento significativo en el salario promedio, que pasó de 37,965 USD en 2014–2016 a 42,401 USD en 2017–2022, con un aumento notable en la desigualdad salarial entre estados. Aunque el número de negocios también se mantiene relativamente estable, existe una alta concentración en unos pocos estados, con valores que van desde apenas unas decenas hasta más de 18,000 establecimientos. Estas cifras reflejan una fuerte heterogeneidad regional, tanto en estructura como en condiciones laborales del sector.

# Visualización de datos

N.B. Prefiero usar Seaborn para visualizar los datos, pero matplotlib es una buena opción también.

## Empleados por negocio

In [ ]:
df_2014_2016 = df_2014_2016.copy()
df_2014_2016["Empleados por negocio"] = (df_2014_2016["Total Population"] / df_2014_2016["Total Number of Establishments"])

# Calcular promedios por estado y ordenar
empleados_por_negocio = (
    df_2014_2016.groupby("State", as_index=False)["Empleados por negocio"]
    .mean()
    .sort_values(by="Empleados por negocio", ascending=False)
)

# Tomar top 10
top_10 = empleados_por_negocio.head(10)

fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(
    data=top_10,
    x="Empleados por negocio",
    y="State",
    ax=ax,
    hue="State",
    palette="viridis"
)

# Mejoras visuales
ax.set_title("Top 10 promedio de empleados por negocio (2014–2016)")
ax.set_xlabel("Empleados por establecimiento")
ax.set_ylabel("Estado")
ax.spines[["top", "right", "bottom"]].set_visible(False)
ax.xaxis.grid(True, linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()


La gráfica muestra los diez estados con mayor promedio de empleados por establecimiento funerario entre 2014 y 2016. Destaca Hawái, con un promedio notablemente más alto que el resto, cercano a 7 empleados por negocio, lo que podría indicar una mayor formalización o concentración de servicios en menos unidades operativas. Le siguen Wyoming y California, ambos por encima de 4 empleados por establecimiento. A partir de Luisiana hacia abajo, el promedio disminuye progresivamente, situándose entre 3 y 4 empleados por negocio.

## Evolución del salario promedio

In [ ]:
# Salario promedio por estado
fig, ax = plt.subplots(figsize=(10, 6))

top_estados = (df_2017_2022.groupby("State")["Average Wage"].mean().sort_values(ascending=False).head(5).index.tolist())
df_top = df_2017_2022[df_2017_2022["State"].isin(top_estados)]

sns.lineplot(data=df_top, x="Year", y="Average Wage", hue="State")

ax.set_title("Top 5 estados con mayor salario promedio (2017–2022)")
ax.set_xlabel("Año")
ax.set_ylabel("Salario promedio (USD)")
ax.legend(title="Estado", loc="upper left", bbox_to_anchor=(1, 1))

plt.ylim(0, 80000)
plt.xlim(2017, 2022)

ax.yaxis.grid(True, linestyle='--')
ax.spines[['top', 'right', 'left']].set_visible(False)

plt.tight_layout()
plt.show()

Se observa una tendencia claramente ascendente en Luisiana, que pasó de tener uno de los salarios más bajos del grupo en 2017 a convertirse en el más alto en 2022, con un aumento sostenido. Delaware también presenta un fuerte crecimiento inicial, aunque se estabiliza después de 2019. En contraste, New Jersey muestra una leve disminución hasta 2020, seguida de una recuperación, mientras que Minnesota mantiene un salario relativamente estable con ligeras variaciones a la baja y al alza. New York, por su parte, presenta un crecimiento más gradual pero constante. En conjunto, la gráfica sugiere que aunque todos los estados mantienen niveles altos, la dinámica de crecimiento salarial varía considerablemente, destacando especialmente el caso de Luisiana como el de mayor aceleración.

## Promedio de negocios funerarios por año

In [ ]:
# Promedio de negocios funerarios por estado
df_grouped = df_2013_2016.groupby("Year")["Total Number of Establishments"].mean().reset_index()
fix, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data=df_grouped, x="Year", y="Total Number of Establishments", marker="o", linewidth=2.5)
ax.set_title("Promedio de negocios funerarios por estado (2013–2016)")
ax.set_ylabel("Promedio de establecimientos")
ax.set_xlabel("Año")
ax.set_xticks([2013, 2014, 2015, 2016])

plt.xlim(2013,2016)
plt.ylim(0,6000)

ax.yaxis.grid(True, linestyle='--')

ax.spines[['top', 'right', 'left']].set_visible(False)

plt.tight_layout()
plt.show()

La gráfica muestra que el promedio de negocios funerarios por estado se mantuvo alto en 2013 y 2014, pero cayó drásticamente a partir de 2015. Esta disminución no necesariamente refleja un colapso en el sector, sino más bien un cambio en la cobertura de los datos: en los primeros años probablemente solo se incluyeron estados con muchos establecimientos, mientras que en 2015 y 2016 se incorporaron más estados con menor presencia, lo que redujo el promedio general.

# Análisis de correlaciones

In [ ]:
# Hacer correlación entre las tres variables
corr_df = df_2014_2016[["Total Population", "Average Wage", "Total Number of Establishments"]]

corr_matrix = corr_df.corr()

# Heatmap de correlaciones
plt.figure(figsize=(12, 6))
sns.heatmap(corr_matrix, annot=True, cmap="Blues", fmt=".2f")
plt.title("Correlación entre variables (2014–2016)")
plt.tight_layout()
plt.show()

La matriz de correlación muestra que la variable con mayor relación es Total Population con Total Number of Establishments, con un coeficiente de 0.54, lo que sugiere una correlación positiva moderada: en general, más población empleada está asociada a una mayor cantidad de negocios funerarios en un estado. En cambio, Average Wage tiene correlaciones muy bajas tanto con la población (0.16) como con el número de establecimientos (0.13), lo que indica que los niveles salariales no están directamente relacionados con el tamaño del sector ni con su estructura operativa.

# Segmentación por población empleada

In [ ]:
# Crear segmentos por terciles (baja, media, alta población)
df_2014_2016["Segmento población"] = pd.qcut(
    df_2014_2016["Total Population"],
    q=3,
    labels=["Baja", "Media", "Alta"]
)

fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(data=df_2014_2016, x="Segmento población", y="Average Wage", palette="viridis", hue="Segmento población", ax=ax)

ax.set_title("Distribución de salario promedio por segmento de población (2014–2016)")
ax.set_xlabel("Segmento de población")
ax.set_ylabel("Salario promedio (USD)")

ax.yaxis.grid(True, linestyle='--')
ax.spines[['top', 'right', 'left']].set_visible(False)

plt.ylim(0, 60000)

plt.tight_layout()
plt.show()

Sorprendentemente, los estados con población baja presentan una mayor dispersión salarial y una mediana más baja, pero también contienen los salarios más altos del conjunto (outliers). En contraste, los estados con población media y alta muestran distribuciones más concentradas, con medianas ligeramente más elevadas que el grupo de baja población, pero sin sobresalir claramente. Esto sugiere que una mayor población empleada no se traduce automáticamente en mejores salarios, y que en estados con menor escala pueden existir negocios más especializados o mejor remunerados.

# Conclusiones

El análisis evidencia que el sector funerario en Estados Unidos presenta una notable heterogeneidad entre estados, tanto en términos de estructura operativa como de condiciones laborales. Entre 2014 y 2016, los estados con mayor número de empleados por establecimiento como Hawái, California y Wyoming podrían indicar modelos de operación más consolidados o una mayor demanda por establecimiento, lo cual resulta relevante al considerar estrategias de escala o eficiencia operativa. Asimismo, entre 2017 y 2022 se observó un crecimiento sostenido en el salario promedio, especialmente en estados como Luisiana, lo que sugiere una evolución positiva en la profesionalización y remuneración del sector. Sin embargo, este crecimiento no está fuertemente correlacionado con el tamaño del empleo ni con el número de negocios, lo que implica que otros factores como la regulación estatal o la especialización de servicios podrían estar incidiendo significativamente. El estudio también reveló que estados con menor población empleada pueden albergar establecimientos con salarios significativamente altos, aunque dispersos, lo cual podría representar oportunidades para introducir servicios premium o diferenciales en mercados menos saturados.

# Recomendaciones

A partir del análisis realizado, se proponen varias recomendaciones estratégicas para una startup interesada en ingresar al sector funerario en Estados Unidos. Primero, se sugiere priorizar mercados como Hawái, California y Wyoming, que presentan un alto promedio de empleados por establecimiento, lo que podría reflejar estructuras operativas más robustas y ofrecer modelos replicables para otros contextos. Además, se identifican oportunidades en estados con menor población laboral en el sector, donde, pese a la escala reducida, existen casos de salarios elevados que podrían indicar nichos de servicios especializados, ideales para propuestas premium o innovadoras. Por otro lado, se recomienda monitorear atentamente las dinámicas salariales regionales como el notable aumento en Louisiana para anticipar impactos en la estructura de costos y ajustar las estrategias de precios. Dado que no se encontró una correlación fuerte entre salario promedio y el tamaño del mercado (en términos de población o número de establecimientos), es importante complementar el análisis con factores contextuales adicionales antes de tomar decisiones de entrada. Finalmente, una estrategia regional escalonada, comenzando por estados con condiciones laborales más predecibles como New York o Minnesota, puede facilitar la validación del modelo de negocio y reducir riesgos antes de expandirse hacia mercados más exigentes o inestables.

# Limitaciones

A pesar de los esfuerzos por recopilar, limpiar y analizar los datos de manera rigurosa, el análisis enfrenta varias limitaciones relevantes. En primer lugar, la cobertura de datos no es uniforme a lo largo del tiempo: mientras que la información sobre establecimientos solo está disponible de 2013 a 2016, los datos de empleo y salario se concentran entre 2014 y 2022, lo que dificulta un análisis longitudinal completo. Además, aunque se trabajó con el grupo industrial 8122 (funeral homes, cemeteries & crematories), no fue posible distinguir entre subsectores específicos como funerarias y crematorios, lo cual limita la precisión del diagnóstico sectorial. También se identificaron problemas de calidad y homogeneidad, especialmente en estados como Alaska, donde se observaron valores faltantes o inconsistentes, lo que llevó a su exclusión del análisis para evitar sesgos. A esto se suma la ausencia de variables contextuales clave como edad de la población, tasas de mortalidad o marcos regulatorios estatales que podrían influir en los resultados observados. Finalmente, debido a las restricciones de tiempo y al carácter introductorio del curso, no se implementaron modelos predictivos avanzados ni técnicas de aprendizaje automático, centrándose únicamente en análisis exploratorios y segmentaciones básicas. Estas limitaciones deben ser tomadas en cuenta al interpretar los hallazgos y derivar recomendaciones estratégicas para la startup interesada en ingresar al mercado funerario en Estados Unidos.

# Anexos

## Código fuente

Todo el análisis fue realizado en un entorno Jupyter Notebook, cuidadosamente estructurado en distintas secciones para facilitar la exploración, limpieza y visualización de los datos. En primer lugar, se instalaron e importaron bibliotecas esenciales como `requests`, `beautifulsoup4`, `pandas`, `numpy`, `matplotlib` y `seaborn`. A continuación, se realizó webscraping desde DataUSA para obtener información clave de la industria funeraria (NAICS 8122), incluyendo el número total de establecimientos, el total de trabajadores (población laboral) y el salario promedio por estado y año. Posteriormente, los datos fueron normalizados, combinados y depurados para eliminar inconsistencias, así como organizados en subconjuntos útiles para distintos periodos temporales. Se aplicaron estadísticas descriptivas como promedios, medianas y desviaciones estándar, seguidas de visualizaciones con gráficas de barras y líneas que permitieron identificar patrones geográficos y temporales relevantes. Además, se llevaron a cabo análisis de correlación entre variables y segmentaciones mediante terciles para categorizar los estados según su población laboral. Todas las visualizaciones fueron generadas con Seaborn sobre Matplotlib y el trabajo se desarrolló íntegramente en un entorno macOS con Python 3.12.

## Archivos CSV

- `funerarias_estados_unidos.csv` — Dataset combinado completo.  
- `funerarias_2013_2016.csv` — Datos de establecimientos.  
- `funerarias_2014_2016.csv` — Datos con empleo y salario.  
- `funerarias_2017_2022.csv` — Datos recientes de empleo y salario.

## Recursos externos

- Fuente de datos: [DataUSA](https://datausa.io/about/usage)  
- Licencia de uso: GNU Affero General Public License v3.0  
- Documentación de Pandas: https://pandas.pydata.org/docs  
- Documentación de Seaborn: https://seaborn.pydata.org/

## Manejo de archivos

El archivo principal utilizado, funerarias_estados_unidos.csv, y los demás se encuentran en la misma carpeta que el presente notebook (.ipynb) y fue generado con codificación UTF-8 utilizando comas (,) como separador. Se recomienda asegurarse de que el entorno de análisis (por ejemplo, Jupyter Notebook) esté configurado para interpretar correctamente estos parámetros. En caso de errores al cargar el archivo, es importante revisar la codificación o delimitación, especialmente si el archivo ha sido editado o transferido entre plataformas.